In [1]:
import sys
sys.path.append('..')  # O ajusta la ruta a la raíz del proyecto
from services.query_builder import QueryBuilder
from WorkFlowAgenteModel import WorkFlowAgenteModel  # Asegúrate de que esto apunte correctamente
from db_connection import DBConnection  # Importamos tu conexión existente
from services.error_handler import ErrorHandler


In [2]:
class WorkFlowAgenteModel:
    def __init__(self, cursor):
        self.query_builder = QueryBuilder(cursor)

    def get_total_procesos(self, modulo):
        """
        Obtiene todos los procesos asociados con un módulo específico.
        Retorna todos los campos relevantes: Id, WorkflowId, Iniciado, Completado, ProductoOrganizacionId.
        """
        self.modulo = modulo  # Guardamos el módulo validado
        self.workflow_id_function = f"wfl.GetModuloAgenteId('{self.modulo}')"

        # Ejecutamos la consulta usando el QueryBuilder para obtener todos los datos importantes
        result = self.query_builder.build_query(
            action="SELECT",
            table="wfl.ProcesoWorkFlow",
            attributes=["Id", "WorkflowId", "ProductoOrganizacionId"],
            conditions={"WorkFlowId": self.workflow_id_function}
        )
        
        return self.query_builder.execute_query(result)

    

    def get_total_productos(self, idcAgente):

        """
        Obtiene los productos necesarios usando el QueryBuilder modificado.
        El modelo construye y ejecuta la query.
        """
        try:
            # Construimos la query
            query = self.query_builder.build_query(
                action="SELECT",
                table="ProductoOrganizacion",
                attributes=["Id", "Codigo", "FichaTecnicaPrincipalId"],  # Solo seleccionamos las columnas necesarias
                conditions={"IdcAgente": idcAgente}
            )

            result = self.query_builder.execute_query(query)
            return result
        except Exception as e:
            ErrorHandler.handle_error(e, query)
            return None

    def get_ficha_tecnica_ids(self, producto_organizacion_ids, modulo_agente_id):

        """
        Obtiene los FichaTecnicaId basados en los ProductoOrganizacionId y ModuloAgenteId usando QueryBuilder.
        El modelo construye y ejecuta la query.
        """
        workflow_id_function = f"wfl.GetModuloAgenteId('{modulo_agente_id}')"
        try:
            # Construimos la query
            query = self.query_builder.build_query(
                action="SELECT",
                table="ftc.FichaTecnica",
                attributes=["Id AS FichaTecnicaId", "Codigo", "ProductoOrganizacionId", "ProcesoWorkFlowId"],
                conditions={"ProductoOrganizacionId": producto_organizacion_ids, "ModuloAgenteId": workflow_id_function}
            )

            result = self.query_builder.execute_query(query)
            return result
        except Exception as e:
            ErrorHandler.handle_error(e, query)
            return None

    def get_productos_procesados(self, idcAgente, producto_organizacion_ids):
        """
        Obtiene los productos procesados que coinciden con los ProductoOrganizacionId y el idcAgente proporcionado.
        """
        # Construimos el WHERE para los ProductoOrganizacionId usando IN
        print(f"dentro funcion: {producto_organizacion_ids}\n")
        try:

            # Ejecutamos la consulta
            query = self.query_builder.build_query(
                action="SELECT",
                table="ProductoOrganizacion",
                attributes=["Id as ProducuctoOrganizacionId", "Codigo"],  # Especificamos lo que queremos obtener
                conditions={"idcAgente": idcAgente, "Id": producto_organizacion_ids}
            )

            result=self.query_builder.execute_query(query)
            return result
        except Exception as e:
            ErrorHandler.handle_error(e, query)
            return None


    def generar_queries(self, producto_organizacion_ids, ficha_tecnica_ids):
        """
        Genera las queries de limpieza basadas en los arrays de ProductoOrganizacionId y FichaTecnicaId usando QueryBuilder.
        El modelo construye las queries y las devuelve.
        """
        queries = []
        try:
            # UPDATE para ProductoOrganizacion
            update_query = self.query_builder.build_query(
                action="UPDATE",
                table="ProductoOrganizacion",
                values={"FichaTecnicaPrincipalId": None, "LastFichaTecnicaPrincipalId": None},
                conditions={"Id": producto_organizacion_ids}  # Usamos IN array
            )
            queries.append(update_query)

            # DELETE para AtributoValorFichaTecnica
            delete_atributo_valor = self.query_builder.build_query(
                action="DELETE",
                table="ftc.AtributoValorFichaTecnica",
                conditions={"FichaTecnicaId": ficha_tecnica_ids}  # Usamos IN array
            )
            queries.append(delete_atributo_valor)

            # DELETE para TechnicalData
            delete_technical_data = self.query_builder.build_query(
                action="DELETE",
                table="ftc.TechnicalData",
                conditions={"FichaTecnicaId": ficha_tecnica_ids}  # Usamos IN array
            )
            queries.append(delete_technical_data)

            # DELETE para RegistrosDeCambios
            delete_registros_cambios = self.query_builder.build_query(
                action="DELETE",
                table="ftc.RegistrosDeCambios",
                conditions={"FichaTecnicaId": ficha_tecnica_ids}  # Usamos IN array
            )
            queries.append(delete_registros_cambios)

            # DELETE para FichaTecnica
            delete_ficha_tecnica = self.query_builder.build_query(
                action="DELETE",
                table="ftc.FichaTecnica",
                conditions={"ProductoOrganizacionId": producto_organizacion_ids}  # Usamos IN array
            )
            queries.append(delete_ficha_tecnica)

            return queries  # Devolvemos las queries generadas

        except Exception as e:
            ErrorHandler.handle_error(e, "Error generando las queries de limpieza")
            return None

In [3]:
db = DBConnection("AemeDb")
db.connect()

# %% Crear una instancia del modelo y probar la consulta
workflow_model = WorkFlowAgenteModel(db.cursor)

# Paso 1: Obtener total de procesos y extraer ProductoOrganizacionId
modulo_test = 'HomologacionProductoLactalis'  # Cambia según sea necesario
total_procesos = workflow_model.get_total_procesos(modulo_test)
# Extraer los ProductoOrganizacionId del resultado
producto_organizacion_ids = [proceso[2] for proceso in total_procesos]  # Cambia según los nombres de las columnas
producto_organizacion_ids = [int(id) for id in producto_organizacion_ids]

#print("ProductosIDs:")
#print(producto_organizacion_ids)
# Paso 2: Obtener productos procesados
idcAgente_test = 45522  # Cambia este número por el idcAgente
productos_procesados = workflow_model.get_productos_procesados(idcAgente_test, producto_organizacion_ids)

# Paso 3: Obtener FichaTecnicaId de los productos procesados
ficha_tecnica_ids = workflow_model.get_ficha_tecnica_ids(producto_organizacion_ids, modulo_test)
ficha_tecnica_ids_extracted = [ficha[0] for ficha in ficha_tecnica_ids]
#print("FichasTecnicas")
#print(ficha_tecnica_ids)
# Paso 4: Generar queries de limpieza
queries_limpieza = workflow_model.generar_queries(producto_organizacion_ids, ficha_tecnica_ids_extracted)

# Mostrar las queries generadas
for query in queries_limpieza:
    print(f"Query generada:\n{query}\n")
db.close()

Conectado a la base de datos: AemeDb
SELECT Id, WorkflowId, ProductoOrganizacionId FROM wfl.ProcesoWorkFlow WHERE WorkFlowId = wfl.GetModuloAgenteId('HomologacionProductoLactalis')
dentro funcion: [154976, 154977, 154984, 154985, 154987, 154989, 154992, 154995, 154978, 154979, 154980, 154981, 154982, 154983, 154986, 154988, 154990, 154991, 200783, 200784, 200785, 200786, 200787, 200788, 200789, 200790, 200791, 200792, 200793, 200794, 200795, 200796, 200797, 200798, 200799, 200800, 200801, 200802, 200803, 200804, 200805, 200806, 200807, 200808, 200809, 200810, 200811, 200812, 200813, 200814, 200815, 200816, 200817, 200818, 200819, 200820, 200821, 200822, 200823, 200824, 200825, 200826, 200827, 200828, 200829, 200830, 200831, 200832, 200833, 200834, 200835, 200836, 200837, 200838, 200839, 200840, 200841, 200842, 200977, 200977, 155215, 155212, 155234, 155235, 155260, 155356, 155357, 155357, 155357, 200869]

Query generada:
UPDATE ProductoOrganizacion SET FichaTecnicaPrincipalId = None, L